In [1]:
import pywikibot
import requests
import sys
import os 
from pathlib import Path
import json
import re 
import time
from datetime import date
from os.path import exists


# https://stackoverflow.com/a/66303932
project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent
sys.path.append(str(project_path))
sys.path.append(str(project_path / 'scripts'))

import scripts.utils.wikidata_utils as wd


In [2]:
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

WIKI_BASE_URL = "https://www.wikidata.org"
WIKI_QUERY_URL = "https://query.wikidata.org"



In [3]:
new_item = pywikibot.ItemPage(repo, title="Q225")
new_item.exists()

True

In [4]:
new_item_json = new_item.toJSON()
# new_item_json

In [5]:

# print(json.dumps(new_item.toJSON(), indent=2, ensure_ascii=False))

In [6]:

start = time.time()
foo = wd.format_display_item(new_item)
stop = time.time()
print(stop-start)




16.983999967575073


In [ ]:


all_ids = wd.get_ids_for_item(new_item, new_item_json, include_pids=True, include_qids=True)
start = time.time()
all_ids_dict = wd.fetch_labels_for_ids(all_ids, lang="en")
stop = time.time()
print('all', stop-start, len(all_ids))


q_ids = wd.get_ids_for_item(new_item, new_item_json, include_pids=False, include_qids=True)
start = time.time()
q_ids_dict = wd.fetch_labels_for_ids(q_ids, lang="en")
stop = time.time()
print('qids', stop-start, len(q_ids))


p_ids = wd.get_ids_for_item(new_item, new_item_json, include_pids=True, include_qids=False)
start = time.time()
p_ids_dict = wd.fetch_labels_for_ids(p_ids, lang="en")
stop = time.time()
print('pids', stop-start, len(p_ids))

all 8.88849687576294 449


In [ ]:
start = time.time()

ids = wd.format_ids_labels(new_item, new_item_json)

stop = time.time()
print('ids', stop-start, len(ids))